# Getting the temperatures

We use the dataset [*Nordic gridded temperature and precipitation data from 1961 to present derived from in-situ observations*](https://cds.climate.copernicus.eu/datasets/insitu-gridded-observations-nordic?tab=overview).

Problem : it is more than 250Go and only a small part of it interests us. This notebook takes care of retrieving and then considering the resulting data.

## Part I: Getting the data.

This is very long and the resulting data is on the git. I suggest you go to part II directly.


In [ ]:
%matplotlib widget

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from urllib.request import urlretrieve
from scipy.spatial import KDTree
from tqdm.notebook import tqdm
from time import time
import netCDF4 as nc
import pandas as pd
import numpy as np
import os

# No temperature data before 1961.
# Data for S1 2024 soon available.
min_year = 1961
max_year = 2023

print("Reading arthropods...", end="")
arth = pd.read_csv("./data/arthropods_sorted.csv", sep = ";")
arth["ymd"] = arth["year"].astype(str) + arth["month"].astype(str).str.zfill(2) + arth["day"].astype(str).str.zfill(2)
print(" Done.")

# The temp grid is always the same.
# We can use the same KDTree for all years to find the close point of the grid fast.
# Conceptually overkill, in practice faster than what I'v done by hand.
print("Constructing KDTree...", end="")
url = f"https://thredds.met.no/thredds/fileServer/ngcd/version_24.03/TG/type1/1961/01/NGCD_TG_type1_version_24.03_19610101.nc"
urlretrieve(url, "19610101.nc")
ds = nc.Dataset(f"19610101.nc", "r")
tree = KDTree(np.array([ds["lat"][:].data.reshape(-1, ), ds["lon"][:].data.reshape(-1, )]).T)
os.remove(f"19610101.nc")
print(" Done.")

# This we will be able to parallelize
def download(chunk):
    row = chunk.iloc[0]
    ymd = row["ymd"]
    year, month, day = ymd[:4], ymd[4:6], ymd[6:]
    url =  f"https://thredds.met.no/thredds/fileServer/ngcd/version_24.03/TG/type1/{year}/{month}/NGCD_TG_type1_version_24.03_{year+month+day}.nc"
    urlretrieve(url, f"{year}/{ymd}.nc")

# This I wasn't able
def read_temp(ymd):
    year = ymd[:4]
    with nc.Dataset(f"{year}/{ymd}.nc", "r") as ds:
        lat = ds["lat"][:].data.reshape(-1, ).copy()
        lon = ds["lon"][:].data.reshape(-1, ).copy()
        temp = ds["TG"][:].data.reshape(-1, ).copy()
        return pd.DataFrame({'lat': lat, 'lon': lon, 'temp': temp})

def extract_save(chunk):
    ymd = chunk.iloc[0]["ymd"]
    weather = read_temp(ymd)
    os.remove(f"{ymd[:4]}/{ymd}.nc")
    f = lambda row: weather.loc[tree.query([row["decimalLatitude"], row["decimalLongitude"]])[1]]["temp"]
    chunk.apply(f, axis = 1).to_csv(f"{ymd[:4]}/{ymd}.csv")

# We work year by year, downloading and then deleting as we go along.
for year in tqdm(range(min_year, max_year + 1), desc="Years"):#, initial = 1961, total=2023 - 1961 + 1):
    if not os.path.exists(str(year)):
        os.mkdir(str(year))

    cur_arth = arth[arth["year"] == year]
    chunks = [cur_arth[cur_arth["ymd"] == ymd] for ymd in tqdm(cur_arth["ymd"].unique(), desc="Chunking", leave=False)]

    with ThreadPoolExecutor() as executor:
        futures = tqdm([executor.submit(download, chunk) for chunk in chunks], desc="Downloading", leave=False)
        
        for future in futures:
            future.result()

    for chunk in tqdm(chunks, desc= "Extracting", leave = False):
        extract_save(chunk)


In [ ]:
# We can then read all the info we have saved and concatenate it.
dfs = []
for ymd in tqdm(arth[(arth["year"] > 1961) & (arth["year"] < 2024)]["ymd"].unique()):
    df = pd.read_csv(f"./{ymd[:4]}/{ymd}.csv", index_col=0)
    dfs.append(df)
temps = pd.concat(dfs) - 273.15 # Kelvin to Celsius
temps.to_csv(f"./data/temps{1961}_{max_year}.csv", index = False)
arthtemp = pd.concat([arth, temps], axis=1)
arthtemp.rename(columns = {'0': "temp"}, inplace = True) # haven't been careful when saving the files
arthtemp.drop(columns = ["ymd"], inplace = True)

# We save it once and for all !
arthtemp.to_csv("./data/arthropods_temps.csv", index = False)

## Part II: Checking the data we've downloaded

In [ ]:
arthtemp.head()

In [ ]:
# Let's see if we can see seasonal patterns
arthtemp.groupby(['year', 'month'])["temp"].mean().plot(figsize = (10, 5))

Clearly something is wrong.

In [ ]:
arthtemp["temp"].describe()

The min value is absurd. Let's investigate.

In [ ]:
len(arthtemp[arthtemp["temp"] < -100]), arthtemp[arthtemp["temp"] < -100]["temp"].unique()

In [ ]:
# This looks like a mistake. Let's remove those few lines.
arthtemp.drop(arthtemp[arthtemp["temp"] < -100].index, inplace = True)

In [ ]:
arthtemp.groupby(['year', 'month'])["temp"].mean().plot(figsize = (10, 5)) # better

In [ ]:
arthtemp["temp"].describe()

In [ ]:
# Generated by ChatGPT
# I needed to check if the temperature read for each observation was correct.
# It wasn't. Now It should be. Will check again.

import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from colorcet import fire

def plot_combined_with_dual_colorbars(df_TG, df_arthropods):
    # Set up a Canvas for plotting
    cvs = ds.Canvas(plot_width=1000, plot_height=600,
                    x_range=(df_TG['lon'].min(), df_TG['lon'].max()),
                    y_range=(df_TG['lat'].min(), df_TG['lat'].max()))

    # Aggregate by TG and shade the result
    agg = cvs.points(df_TG, 'lon', 'lat', ds.mean('TG'))
    base_img = tf.shade(agg, cmap=fire, how='log')
    base_img_rgb = tf.set_background(base_img, "white").to_pil()  # Convert to RGBA

    # Plot with Matplotlib
    fig, ax = plt.subplots(figsize=(9, 4))
    
    # Set extent to match the coordinates of the data
    extent = [df_TG['lon'].min(), df_TG['lon'].max(), df_TG['lat'].min(), df_TG['lat'].max()]
    
    # Display the Datashader image with proper extent
    ax.imshow(base_img_rgb, extent=extent)

    # Scatter plot of arthropod observations
    sc = ax.scatter(df_arthropods['lon'], df_arthropods['lat'], c=df_arthropods['temperature'],
                    cmap=mpl.colors.ListedColormap(fire), edgecolor=None, s=10, alpha=0.7)

    # Create a colorbar for the arthropod temperature
    cbar_arthropod = plt.colorbar(sc, ax=ax, orientation='vertical', pad=0.02, shrink=0.7)
    cbar_arthropod.set_label("Arthropod Temperature Observations")

    # Create a second colorbar for the background temperature (TG)
    norm = mpl.colors.Normalize(vmin=df_TG['TG'].min(), vmax=df_TG['TG'].max())
    sm = mpl.cm.ScalarMappable(cmap=mpl.colors.ListedColormap(fire), norm=norm)
    sm.set_array([])  # Dummy array for the colorbar
    cbar_background = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.08, shrink=0.7)
    cbar_background.set_label("Background Temperature (TG)")

    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.title("Temperature Field with Arthropod Observations Overlay")
    plt.show()

# Call the function
df_day = df[df["ymd"] == "2021-7-1"]
plot_combined_with_dual_colorbars(weather20210701, df_day)